In [ ]:
import langchain

In [ ]:
import dotenv

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:
api_key = os.getenv("GROQ_API_KEY", "No key")
ls = [1,2,3]

In [ ]:
import logging
import colorlog
import os
from dotenv import load_dotenv

load_dotenv()
def setup_logger(name: str = __name__) -> logging.Logger:
    """
    Set up a colored logger with consistent formatting
    """
    logger = logging.getLogger(name)
    
    # Prevent duplicate handlers
    if logger.handlers:
        return logger
    
    # Set log level from environment or default to INFO
    log_level = os.getenv('LOG_LEVEL', 'INFO').upper()
    logger.setLevel(getattr(logging, log_level))
    
    # Create console handler with color formatting
    handler = colorlog.StreamHandler()
    
    # Color formatter
    formatter = colorlog.ColoredFormatter(
        '%(log_color)s%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        log_colors={
            'DEBUG': 'cyan',
            'INFO': 'green',
            'WARNING': 'yellow',
            'ERROR': 'red',
            'CRITICAL': 'red,bg_white',
        }
    )
    
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    
    return logger
# Create a default logger instance
logger = setup_logger('chatbot')

In [ ]:
import os
from dotenv import load_dotenv
#from logger_config import setup_logger

load_dotenv()
logger = setup_logger(__name__)
class Config:
    """Configuration class for the chatbot application"""
    
    # API Configuration
    GROQ_API_KEY = os.getenv("GROQ_API_KEY")
    
    # Model Configuration
    MODEL_NAME = os.getenv('MODEL_NAME', 'mixtral-8x7b-32768')
    TEMPERATURE = float(os.getenv('TEMPERATURE', 0.7))
    MAX_TOKENS = int(os.getenv('MAX_TOKENS', 1024))
    
    # Application Configuration
    APP_NAME = os.getenv('APP_NAME', 'LangChain Groq Chatbot')
    
    @classmethod
    def validate_config(cls):
        """Validate that all required configuration is present"""
        if not cls.GROQ_API_KEY:
            logger.error("GROQ_API_KEY is not set in environment variables")
            raise ValueError("GROQ_API_KEY is required")
        
        logger.info("Configuration validated successfully")
        return True
# Validate configuration on import
Config.validate_config()

In [ ]:
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage, SystemMessage
#from config import Config
#from logger_config import setup_logger
import streamlit as st

logger = setup_logger(__name__)
class GroqChatbot:
    """A simple chatbot using LangChain and Groq API"""
    
    def __init__(self):
        """Initialize the chatbot with Groq LLM"""

        print(f"GROQ KEY: {Config.GROQ_API_KEY}")
        try:
            self.llm = ChatGroq(
                groq_api_key= Config.GROQ_API_KEY,
                model_name="llama-3.1-8b-instant",
                temperature=Config.TEMPERATURE,
                max_tokens=Config.MAX_TOKENS
            )
            
            self.system_message = SystemMessage(
                content="You are a helpful AI assistant. Provide clear, "
                       "concise, and helpful responses to user questions."
            )
            
            logger.info(f"Chatbot initialized with model: {Config.MODEL_NAME}")
            
        except Exception as e:
            logger.error(f"Failed to initialize chatbot: {e}")
            raise
    
    def get_response(self, user_input: str) -> str:
        """
        Get response from the chatbot for user input
        
        Args:
            user_input (str): User's message
            
        Returns:
            str: Chatbot's response
        """
        try:
            messages = [
                self.system_message,
                HumanMessage(content=user_input)
            ]
            
            logger.info(f"Processing user input: {user_input[:100]}...")
            response = self.llm(messages)
            logger.info("Response generated successfully")
            
            return response.content
            
        except Exception as e:
            logger.error(f"Error generating response: {e}")
            return "I apologize, but I encountered an error processing your request."
def main():
    """Main function for command-line interface"""
    logger.info("Starting LangChain Groq Chatbot")
    
    try:
        chatbot = GroqChatbot()
        
        print(f"\n🤖 {Config.APP_NAME}")
        print("=" * 50)
        print("Type 'quit', 'exit', or 'bye' to end the conversation\n")
        
        while True:
            user_input = input("You: ").strip()
            
            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("\n👋 Goodbye! Thanks for chatting!")
                break
            
            if not user_input:
                continue
            
            response = chatbot.get_response(user_input)
            print(f"\n🤖 Bot: {response}\n")
            
    except KeyboardInterrupt:
        print("\n\n👋 Goodbye! Thanks for chatting!")
    except Exception as e:
        logger.error(f"Application error: {e}")
        print(f"An error occurred: {e}")
if __name__ == "__main__":
    main()